In [2]:
import anndata
import numpy as np
import pandas as pd


n_obs = 500
n_vars = 50
n_pca = 10

X_data = np.random.rand(n_obs, n_vars)

my_counts = np.random.rand(n_obs, n_vars)

X_pca = np.random.rand(n_obs, n_pca)

cell_lines = np.random.choice(["cell_line_a", "cell_line_b", "cell_line_c"], n_obs)
dosages = np.random.choice([10.0, 100.0, 1000.0], n_obs)
drugs = ["drug_a", "drug_b", "drug_c", "Vehicle"]
drug1 = np.random.choice(drugs, n_obs)
drug2 = np.random.choice(drugs, n_obs)
drug3 = np.random.choice(drugs, n_obs)

obs_data = pd.DataFrame(
    {"cell_type": cell_lines, "dosage": dosages, "drug1": drug1, "drug2": drug2, "drug3": drug3}
)

# Create an AnnData object
adata = anndata.AnnData(X=X_data, obs=obs_data)

adata.uns["cell_flow_conditions"] = {}

# Add the random data to .layers and .obsm
adata.layers["my_counts"] = my_counts
adata.obsm["X_pca"] = X_pca

for col in adata.obs.columns:
    adata.obs[col] = adata.obs[col].astype("category")

drug_emb = {}
for drug in adata.obs["drug1"].cat.categories:
    drug_emb[drug] = np.random.randn(5, 1)
adata.uns[UNS_KEY_CONDITIONS]["drug"] = drug_emb

for drug in adata.obs["cell_type"].cat.categories:
    drug_emb[drug] = np.random.randn(3, 1)
adata.uns[UNS_KEY_CONDITIONS]["cell_type"] = drug_emb


/Users/dominikklein/miniforge3/envs/cfp/lib/python3.11/site-packages/anndata/_core/aligned_df.py:67: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [85]:
import itertools
from collections.abc import Callable, Sequence
from dataclasses import dataclass
from typing import Any, Literal, Optional

import anndata
import jax
import jax.numpy as jnp
import numpy as np
import pandas as pd
import scipy.sparse as sp
from tqdm import tqdm

from cfp._constants import CONTROL_HELPER, UNS_KEY_CONDITIONS


@dataclass
class PerturbationData:
    cell_data: jax.Array  # (n_cells, n_features)
    split_covariates_mask: jax.Array  # (n_cells,), which cell assigned to which source distribution
    split_covariates_to_idx: dict[str, int]  # (n_sources,) dictionary explaining split_covariates_mask
    perturbation_covariates_mask: jax.Array  # (n_cells,), which cell assigned to which target distribution
    perturbation_covariates_to_idx: dict[str, int]  # (n_targets,), dictionary explaining perturbation_covariates_mask
    condition_data: jax.Array  # (n_targets,) all embeddings for conditions
    control_to_perturbation: dict[int, jax.Array]  # mapping from control idx to target distribution idcs

    @property
    def n_controls(self) -> int:
        """Returns the number of control covariate values."""
        return len(self.split_covariates_to_idx)

    @property
    def n_perturbed(self) -> int:
        """Returns the number of perturbation covariate combinations."""
        return len(self.perturbation_covariates_to_idx)

    def _format_params(self, fmt: Callable[[Any], str]) -> str:
        params = {"n_controls": self.n_controls, "n_perturbed": self.n_perturbed}
        return ", ".join(f"{name}={fmt(val)}" for name, val in params.items())

    def __repr__(self) -> str:
        return f"{self.__class__.__name__}[{self._format_params(repr)}]"


def _get_cell_data(adata: anndata.AnnData, cell_data: Literal["X"] | dict[str, str]) -> jax.Array:
    if cell_data == "X":
        cell_data = adata.X
        if isinstance(cell_data, sp.csr_matrix):
            cell_data = jnp.asarray(cell_data.toarray())
        else:
            cell_data = jnp.asarray(cell_data)
    else:
        assert isinstance(cell_data, dict)
        assert "attr" in cell_data
        assert "key" in cell_data
        cell_data = jnp.asarray(getattr(adata, cell_data["attr"])[cell_data["key"]])
    return cell_data


def _verify_control_data(adata: anndata.AnnData, data: tuple[str, Any]):
    assert isinstance(data, tuple)
    assert len(data) == 2
    assert data[0] in adata.obs
    assert isinstance(adata.obs[data[0]].dtype, pd.CategoricalDtype)
    assert data[1] in adata.obs[data[0]].values


def _check_shape(arr: float | np.ndarray) -> np.ndarray:
    if not hasattr(arr, "shape") or len(arr.shape) == 0:
        return np.ones((1, 1)) * arr

    if arr.ndim == 1:
        return arr[:, None]
    elif arr.ndim == 2:
        if arr.shape[0] == 1:
            return arr
        if arr.shape[1] == 1:
            return np.transpose(arr)
        raise ValueError("TODO, wrong shape.")
    elif arr.ndim > 2:
        raise ValueError("TODO. Too many dimensions.")

    raise ValueError("TODO. wrong data for embedding.")


def _get_perturbation_covariates(
    adata: anndata.AnnData,
    embedding_dict: dict[str, dict[str, np.ndarray]],
    perturbation_covariates: Sequence[tuple[str, str | None]],
    perturbation_covariate_combinations: Optional[Sequence[Sequence[str]]],
) -> dict:
    if len(perturbation_covariate_combinations) > 1:
        raise NotImplementedError("Currently only one group of perturbation covariates is supported.")
    max_combo_length = 1 if len(perturbation_covariate_combinations)==0 else max([len(group) for group in perturbation_covariate_combinations])

    single_embeddings = []
    for obs_col in perturbation_covariates:
        values = list(adata.obs[obs_col[0]].unique())
        if len(values) != 1:
            raise ValueError("Too many categories within distribution found")

        if obs_col[1] in embedding_dict:
            assert isinstance(adata.uns[UNS_KEY_CONDITIONS][obs_col[1]], dict)
            vals = adata.obs[obs_col[0]].values
            assert len(np.unique(vals)) == 1
            assert vals[0] in embedding_dict[obs_col[1]]
            arr = jnp.asarray(embedding_dict[obs_col[1]][vals[0]])
            arr = _check_shape(arr)
            single_embeddings.append(arr)
        elif obs_col[1] is None:
            single_embeddings.append(_check_shape(values[0]))
        else:
            arr = jnp.asarray(adata.obs[obs_col[1]].values[0])
            arr = _check_shape(arr)
            single_embeddings.append(arr)
    conds_no_combination = jnp.tile(jnp.concatenate(single_embeddings, axis=-1), (max_combo_length, 1))


    combo_embeddings = []
    if len(perturbation_covariate_combinations)> 0:
        
        for obs_col in perturbation_covariate_combinations[0]:
            values = list(adata.obs[obs_col].unique())
            if len(values) != 1:
                raise ValueError("Too many categories within distribution found")
            if obs_col in embedding_dict:
                assert isinstance(adata.uns[UNS_KEY_CONDITIONS][obs_col], dict)
                vals = adata.obs[obs_col].values
                assert len(np.unique(vals)) == 1
                assert vals[0] in embedding_dict[obs_col]
                arr = jnp.asarray(embedding_dict[obs_col][vals[0]])
                arr = _check_shape(arr)
                combo_embeddings.append(arr)
            else:
                arr = jnp.asarray(adata.obs[obs_col].values[0])
                arr = _check_shape(arr)
                combo_embeddings.append(arr)
        conds_combination = jnp.concatenate(combo_embeddings, axis=0) 
    else:
        conds_combination = []
    
    to_concat = []
    if len(conds_no_combination) > 0:
        to_concat.append(conds_no_combination)
    if len(conds_combination) > 0:
        to_concat.append(conds_combination)

    conds = jnp.concatenate(to_concat, axis=-1)
    return conds
    


def load_from_adata(
    adata: anndata.AnnData,
    cell_data: Literal["X"] | dict[str, str],
    control_data: tuple[str, Any],
    split_covariates: Sequence[tuple[str, str | None]],
    perturbation_covariates: Sequence[tuple[str, str | None]],
    perturbation_covariate_combinations: Optional[Sequence[Sequence[str]]],
) -> PerturbationData:
    """Load cell data from an AnnData object.

    Args:
        adata: An :class:`~anndata.AnnData` object.
        cell_data: Where to read the cell data from. If of type :class:`dict`, the key
            "attr" should be present and the value should be an attribute of :class:`~anndata.AnnData`.
            The key `key` should be present and the value should be the key in the respective attribute
        control_data: Tuple of length 2 with first element defining the column in :class:`~anndata.AnnData`
          and second element defining the value in `adata.obs[control_data[0]]` used to define the source
          distribution.
        split_covariates: Covariates in adata.obs defining the control distribution.
        perturbation_covariates: Covariates in adata.obs characterizing the source distribution (together
          with `split_covariates`).
          First of a tuple is a column name in adata.obs, the second element is the key in
          adata.uns[`UNS_KEY_CONDITION`] if it exists, otherwise the obs column name if the
          embedding of the covariate should directly be read from it.
        perturbation_covariate_combinations: Groups of covariates that should be combined and treated as
          an unordered set.

    Returns
    -------
        PerturbationData: Data container for the perturbation data.
    """
    # TODO(@MUCDK): add device to possibly only load to cpu
    if split_covariates is None or len(split_covariates) == 0:
        adata.obs[CONTROL_HELPER] = True
        adata.obs[CONTROL_HELPER] = adata.obs[CONTROL_HELPER].astype("category")
        split_covariates = [CONTROL_HELPER]
    _verify_control_data(adata, control_data)
    if len(perturbation_covariate_combinations) > 1:
        raise NotImplementedError("Currently only one group of perturbation covariates is supported.")

    if UNS_KEY_CONDITIONS not in adata.uns:
        adata.uns[UNS_KEY_CONDITIONS] = {}

    for covariate in split_covariates:
        assert covariate in adata.obs
        assert adata.obs[covariate].dtype.name == "category"

    src_dist = {covariate: adata.obs[covariate].cat.categories for covariate in split_covariates}
    src_counter = 0
    tgt_counter = 0
    src_dists = list(itertools.product(*src_dist.values()))

    control_to_perturbation = {}
    cell_data = _get_cell_data(adata, cell_data)
    split_covariates_mask = np.full((adata.n_obs,), -1, dtype=jnp.int32)
    split_covariates_to_idx = {}
    perturbation_covariates_mask = np.full((adata.n_obs,), -1, dtype=jnp.int32)
    perturbation_covariates_to_idx = {}
    condition_data = []

    control_mask = (adata.obs[control_data[0]] == control_data[1]) == 1
    for src_combination in tqdm(src_dists):
        filter_dict = dict(zip(split_covariates, src_combination, strict=False))
        split_cov_mask = (adata.obs[list(filter_dict.keys())] == list(filter_dict.values())).all(axis=1)
        mask = split_cov_mask * control_mask 
        if mask.sum() == 0:
            continue
        control_to_perturbation[src_counter] = []
        split_covariates_mask[mask] = src_counter
        split_covariates_to_idx[src_counter] = src_combination

        tgt_dist = {covariate[0]: adata.obs[covariate[0]].cat.categories for covariate in perturbation_covariates}
        conditional_distributions = []
        for tgt_combination in itertools.product(*tgt_dist.values()):
            # TODO check whether we have the split covariances included here. don't think we did before
            filter_dict_tgt = {
                covariate[0]: value for covariate, value in zip(perturbation_covariates, tgt_combination, strict=False)
            }
            mask = ((adata.obs[list(filter_dict_tgt.keys())] == list(filter_dict_tgt.values())).all(axis=1) * (
                1 - control_mask
            ) * split_cov_mask) ==1
            if mask.sum() == 0:
                continue
            conditional_distributions.append(tgt_counter)
            perturbation_covariates_mask[mask]=tgt_counter
            perturbation_covariates_to_idx[tgt_counter] = tgt_combination
            control_to_perturbation[src_counter] = tgt_counter
            embedding = _get_perturbation_covariates(
                adata[mask], adata.uns[UNS_KEY_CONDITIONS], perturbation_covariates, perturbation_covariate_combinations
            )
            condition_data.append(embedding)
            tgt_counter += 1
        control_to_perturbation[src_counter] = np.array(conditional_distributions)
        src_counter += 1
    condition_data = jnp.array(condition_data)

    return PerturbationData(
        cell_data=cell_data,
        split_covariates_mask=split_covariates_mask,
        split_covariates_to_idx=split_covariates_to_idx,
        perturbation_covariates_mask=perturbation_covariates_mask,
        perturbation_covariates_to_idx=perturbation_covariates_to_idx,
        condition_data=condition_data,
        control_to_perturbation=control_to_perturbation,
    )


In [86]:
control_covariates = ["cell_type"]
control_data = ("drug1", "Vehicle")
perturbation_covariates = [("drug1", "drug"), ("cell_type", "cell_type")]

In [87]:
pdata = load_from_adata(adata, cell_data="X", control_data=control_data, split_covariates = control_covariates, perturbation_covariates=perturbation_covariates, perturbation_covariate_combinations=[])

100%|██████████| 3/3 [00:00<00:00, 84.14it/s]


In [88]:
np.unique(pdata.perturbation_covariates_mask)

array([-1,  0,  1,  2,  3,  4,  5,  6,  7,  8], dtype=int32)

In [89]:
class CFSampler:
    """Data sampler for :class:`~cfp.data.data.PerturbationData`.

    Parameters
    ----------
    data : PerturbationData
        The data object to sample from.
    batch_size : int
        The batch size.
    """

    def __init__(self, data: PerturbationData, batch_size: int = 64):
        self.data = data
        self.batch_size = batch_size
        self.n_source_dists = data.n_controls
        self.n_target_dists = data.n_perturbed

        def _sample(rng: jax.Array) -> Any:
            rng_1, rng_2, rng_3, rng_4 = jax.random.split(rng, 4)
            source_dist_idx = jax.random.randint(rng_1, [1], 0, self.n_source_dists).item()
            source_cells = self.data.cell_data[self.data.split_covariates_mask == source_dist_idx]
            source_batch = jax.random.choice(rng_2, source_cells, (self.batch_size,), replace=True)
            target_dist_idx = jax.random.randint(
                rng_3, [1], 0, self.data.control_to_perturbation[source_dist_idx].shape[0]
            ).item()
            target_cells = self.data.cell_data[self.data.perturbation_covariates_mask == target_dist_idx]
            target_batch = jax.random.choice(rng_4, target_cells, (self.batch_size,), replace=True)
            condition_batch = jnp.tile(self.data.condition_data[target_dist_idx], (self.batch_size, 1))

            return {
                "src_lin": source_batch,
                "tgt_lin": target_batch,
                "src_condition": condition_batch,
            }

        self.sample = _sample


In [90]:
s = CFSampler(pdata, batch_size=64)

In [91]:
out = s.sample(jax.random.PRNGKey(42))

In [92]:
out["src_lin"].shape, out["tgt_lin"].shape, out["src_condition"].shape


((64, 50), (64, 50), (64, 8))

Now with combinations of drugs

In [93]:
control_covariates = ["cell_type"]
control_data = ("drug1", "Vehicle")
perturbation_covariates = [("drug1", "drug"), ("drug2", "drug"), ("cell_type", "cell_type")]
perturbation_covariate_combinations = [["drug1", "drug2"]]

In [94]:
pdata = load_from_adata(adata, cell_data="X", control_data=control_data, split_covariates = control_covariates, perturbation_covariates=perturbation_covariates, perturbation_covariate_combinations=perturbation_covariate_combinations)

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]


TypeError: Value 'drug_a' with dtype <U6 is not a valid JAX array type. Only arrays of numeric types are supported by JAX.

pdata = load_from_adata(adata, cell_data="X", control_data=control_data, split_covariates = control_covariates, perturbation_covariates=perturbation_covariates, perturbation_covariate_combinations=perturbation_covariate_combinations)